This notebook will be used to develop and compare regression models to correlate the eHydro bathymetric surveys with cloud-masked Sentinel-2 surface refelctances. These models will hopefully provide USACE and the eHydro program with a new, robust, accurate tool for unmanned bathymetric estiamtes. This will be possible at 10-meter resolution at a frequency of up to 5 days.
- First starting with XGBoost, RF, and SVM-RBF regressors in the SWG. May try some NN as well
- band maths here with the green and blue bands (short wavelengths penetrate water columns more)
- include some metadata (AD, CX, BD? Single vs dual beam?)? Will look into more that may be beneficial

In [1]:
import os
import re
import pickle
import rasterio
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Functions

In [13]:
# Function to normalize an array
def normalize(array):
    return (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array))

# Function to read .tif files
def read_tif(file_path):
    with rasterio.open(file_path) as src:
        array = src.read()  # Read all bands
        profile = src.profile  # Get metadata (optional, for reference)
    return array, profile

def extract_raster_data(pair_tuple):
    images_data = []

    for paths in pair_tuple:
        bathy_path = paths[1]
        s2_path = paths[0]

        # --- Step 1: Open Bathymetry Raster ---
        with rasterio.open(bathy_path) as bathy:
            bathy_data = bathy.read(1)  # Bathymetry data (band 1)
            bathy_nodata = bathy.nodata  # NoData value
            bathy_transform = bathy.transform
            bathy_shape = bathy.shape

        # --- Step 2: Open Sentinel-2 Raster ---
        with rasterio.open(s2_path) as s2:
            if s2.shape != bathy_shape or s2.transform != bathy_transform:
                raise ValueError(
                    f"Inconsistent shapes or transforms:\n"
                    f"Bathymetry Shape: {bathy_shape}, Sentinel-2 Shape: {s2.shape}.\n"
                    f"Bathymetry Transform: {bathy_transform}, Sentinel-2 Transform: {s2.transform}.\n"
                    f"Ensure rasters have identical extents and resolutions."
                )

            # Read Sentinel-2 bands
            bands = {
                "red": normalize(s2.read(3)),
                "green": normalize(s2.read(2)),
                "blue": normalize(s2.read(1)),
                "nir": normalize(s2.read(4))
            }

            s2_nodata = s2.nodata  # Sentinel-2 NoData value

        # --- Step 3: Flatten Bands ---
        flat_bathy = bathy_data.flatten()
        flat_bands = {key: band.flatten() for key, band in bands.items()}

        # --- Step 4: Mask NoData Values ---
        valid_mask = (
            ~np.isnan(flat_bathy) &  # Valid bathy pixels
            (flat_bathy != bathy_nodata)  # Exclude bathy NoData
        )

        for band in flat_bands.values():
            valid_mask &= (band != s2_nodata)  # Exclude Sentinel-2 NoData

        # Apply the mask
        valid_bathy = flat_bathy[valid_mask].reshape(-1, 1)  # Reshape bathy to (n_pixels, 1)
        valid_features = np.column_stack([band[valid_mask] for band in flat_bands.values()])

        # --- Step 5: Combine Features and Targets ---
        # combined_features = np.concatenate((valid_bathy, valid_features), axis=1)  # Combine bathy and S2
        images_data.append((valid_features, valid_bathy.flatten()))  # Flatten bathy for targets

    return images_data

def prepare_data(pairs):
    X = []
    y = []
    for features, targets in pairs:
        X.append(features[:,1:])  # Keep features from this pair
        y.append(targets)   # Keep corresponding targets
    return np.vstack(X), np.hstack(y)

def survey_name_type(surveynames):
    """
    Will take in the list of surveynames and extract the NCF channel ID and the survey type
    """
    surveytypes = ['AD', 'BD', 'CS', 'PA', 'PR', 'XA', 'XB', 'XC', 'OT', 'DS']

    extracted_parts = [re.match(r'^(.*?)_\d{8}', path).group(1) for path in surveynames if re.match(r'^(.*?)_\d{8}', path)]
    channel_ids = [re.sub(r'^.*?_DIS_', '', path) for path in extracted_parts]

    isolated_survey_types = []
    for path in surveynames:
        for type in surveytypes:
            if type in path:
                isolated_survey_types.append(type)
                break  # Stop checking after the first match

    return channel_ids, isolated_survey_types

def create_composite_bands_with_existing(flattened_s2):
    if flattened_s2.shape[1] != 4:
        raise ValueError("Input array must have 4 columns representing B, G, R, NIR bands.")

    # Split the bands
    blue = flattened_s2[:, 0]
    green = flattened_s2[:, 1]

    # Compute composite bands
    with np.errstate(divide='ignore', invalid='ignore'):
        bluegreen = np.divide(blue, green, out=np.zeros_like(blue), where=green != 0)
        greenblue = np.divide(green, blue, out=np.zeros_like(green), where=blue != 0)
        stumpf = np.divide(
            np.log(blue + 1e-6), np.log(green + 1e-6), out=np.zeros_like(blue), where=(green > 0) & (blue > 0)
        )

    # Normalize composite bands
    bluegreen = normalize(bluegreen)
    greenblue = normalize(greenblue)
    stumpf = normalize(stumpf)

    # Combine all bands
    combined_array = np.hstack((flattened_s2, bluegreen[:, None], greenblue[:, None], stumpf[:, None]))

    return combined_array

def get_pixel_positions(raster_path):
    with rasterio.open(raster_path) as src:
        # Get the affine transformation of the raster
        transform = src.transform
        
        # Read the first band to determine valid (non-NaN) pixels
        band_data = src.read(1, masked=True)  # Read the first band as a masked array
        valid_mask = ~band_data.mask          # Valid pixels where mask is False

        # Get raster dimensions
        height, width = src.height, src.width

        # Create arrays of pixel indices
        row_indices, col_indices = np.meshgrid(np.arange(height), np.arange(width), indexing="ij")

        # Compute x, y positions using the affine transform
        xs, ys = rasterio.transform.xy(transform, row_indices, col_indices, offset='center')
        xs = np.array(xs).flatten()
        ys = np.array(ys).flatten()

        # Filter x, y positions to include only valid pixels
        valid_positions = np.column_stack((xs[valid_mask.flatten()], ys[valid_mask.flatten()]))

    return valid_positions

def prepare_train_data(surveynames):
    images = [(os.path.join(S2_PATH, f'{name}.tif'), os.path.join(BATHY_PATH, f'{name}.tif')) for name in surveynames]
    images_data = extract_raster_data(images)

    good_pairs = []
    goodnames = []
    for name, pairs in zip(surveynames, images_data):
        if pairs[0].shape[0] != 0:
            good_pairs.append(pairs)
            goodnames.append(name)

    ncf_channels, survey_types = survey_name_type(goodnames)
    all_bands = [create_composite_bands_with_existing(pair[0]) for pair in good_pairs]
    pixel_positions = [get_pixel_positions(os.path.join(S2_PATH, f'{name}.tif')) for name in goodnames]
    
    data = {}

    # Iterate through the names and corresponding data
    for i, name in enumerate(goodnames):
        # Extract data for the current iteration
        bands = all_bands[i]               # Shape (n_pixels, 7)
        positions = pixel_positions[i]     # Shape (n_pixels, 2)
        bathymetry = good_pairs[i][1]      # Shape (n_pixels,)

        # Create a dataframe directly using a dictionary comprehension
        data[name] = pd.DataFrame({
            "Blue": bands[:, 0],
            "Green": bands[:, 1],
            "Red": bands[:, 2],
            "NIR": bands[:, 3],
            "Blue/Green": bands[:, 4],
            "Green/Blue": bands[:, 5],
            "Stumpf": bands[:, 6],
            "X": positions[:, 0],
            "Y": positions[:, 1],
            "Channel_Name": [ncf_channels[i]] * len(bands),  # Repeating value directly
            "Bathymetry": bathymetry
        })

    combined_df = pd.concat(data.values(), ignore_index=True)

    encoder = LabelEncoder()
    combined_df['Channel_Name_Encoded'] = encoder.fit_transform(combined_df['Channel_Name'])

    output = open(os.path.join(WORK_DIR, 'Channel_Name_label_encoders.pkl'), 'wb')
    pickle.dump(encoder, output)
    output.close()
    
    # Drop original categorical columns
    combined_df.drop(columns=['Channel_Name'], inplace=True)

    X = combined_df.drop(columns=['Bathymetry'])
    y = combined_df['Bathymetry']

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

    # Split temp into validation (15%) and test (15%)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    return data, combined_df, X_train, y_train, X_test, y_test, X_val, y_val

# Establish working directories

In [10]:
# your work directory
WORK_DIR = '/home/clay/Documents/SDB/CESWG'

S2_PATH =os.path.join(WORK_DIR, 'processed/S2')
BATHY_PATH = os.path.join(WORK_DIR,'processed/Bathy')

In [11]:
surveynames = os.listdir(BATHY_PATH)
surveynames = [f[:-4] for f in os.listdir(BATHY_PATH) if f.endswith('.tif')]

# Sentinel-2 band manipulation and including other data (good reference is Chybicki et al. 2023)
- Blue Green ratios (Blue/Green, Green/Blue, looking for others)
- Stumpf log ratio of blue green (https://aslopubs.onlinelibrary.wiley.com/doi/10.4319/lo.2003.48.1_part_2.0547)
- Coordinates
- NCF channel ID name
- Survey type
- Spectral indices?? (NDVI and NDWI, NDWI could make sense but would leave this to the end)


The Chybicki 2023 paper had models perform extremely well when including all bands, the Stumpf log ratio, and the UTM coordinates. I think including more blue-green ratios and the survey type as well will increase my accuracy

In [ ]:
all_data, combined_data, X_train, y_train, X_test, y_test, X_val, y_val = prepare_train_data(surveynames)

# Prepare data for test_train_split
- Trains per pixel


- k-fold segmentation for training?
- try 3 regression models for now: SVM, RF, and XGBoost
- may try ElasticNet from cuML, and some shallow NNs

In [ ]:
# Confirm dataset sizes
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Testing set size:", X_test.shape[0])

1. RF Regression:

After 191min of training (initial run)
- R2 Score: 0.9818
- RMSE: 2.1344      (ft)
- MAE: 1.0686       (ft)

In [ ]:
# Random Forest Model
rf_model = RandomForestRegressor(
    n_estimators=100, 
    criterion='squared_error',  
    min_samples_split=4, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='sqrt',                   # 'sqrt', 'log2', int, or float
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=-1, 
    random_state=42, 
    verbose=0, 
    warm_start=False, 
    ccp_alpha=0.0, 
    max_samples=None, 
    monotonic_cst=None
)

rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

r2 = r2_score(y_test, rf_predictions)
rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
mae = mean_absolute_error(y_test, rf_predictions)
print(f"R2 Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
val_predictions = rf_model.predict(X_val)

r2 = r2_score(y_val, val_predictions)
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
mae = mean_absolute_error(y_val, val_predictions)
print(f"R2 Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

2. XGBoost Regression

n_estimators=500, learning_rate=0.3, max_depth=10, grow_policy= 'lossguide', booster= 'gbtree',:
- R2 Score= 0.8529
- RMSE= 6.0626
- MAE= 3.9175

In [ ]:
# Define and configure the XGBoost regressor
xgb_model = XGBRegressor(
    n_estimators=500,      # Number of trees
    learning_rate=0.3,     # Learning rate
    max_depth=10,           # Maximum tree depth
    grow_policy= 'lossguide',
    booster= 'gbtree',
    random_state=42        # Random seed for reproducibility
)

# Train the model
xgb_model.fit(
    X_train, y_train
)

In [ ]:
xgb_predictions = xgb_model.predict(X_test)

r2 = r2_score(y_test, xgb_predictions)
rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
mae = mean_absolute_error(y_test, xgb_predictions)
print(f"R2 Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
val_predictions = xgb_model.predict(X_val)

r2 = r2_score(y_val, val_predictions)
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
mae = mean_absolute_error(y_val, val_predictions)
print(f"R2 Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

3. SVM regression:
- R2 Score: 
- RMSE: 
- MAE: 

In [ ]:
# Support Vector Machine Model
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)


r2 = r2_score(y_test, svm_predictions)
rmse = np.sqrt(mean_squared_error(y_test, svm_predictions))
mae = mean_absolute_error(y_test, svm_predictions)
print(f"R2 Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")